In [2]:
import numpy as np
import tensorflow as tf

In [ ]:
# the model itself
def cnn_clocator(inputs, classes, mode):
    
    conv1 = tf.layers.conv2d(inputs=inputs, filters=50, kernel_size=[3,3], padding="same", activation=tf.nn.relu)
    
    conv2 = tf.layers.conv2d(inputs=conv1, filters=80, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
    
    pool = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool_flat = 